# This notebook serves as an example of using Google's API.v3 for simple manipulations on files on Drive
## as well as
# uploading pandas.DataFrame to Google Sheets

### This notebook is delivered with proper nb_conda env.
### Once the env is created, it should appear in Kernel -> Change Kernel menu.
### If it does not - visit [nb_conda github repo](https://github.com/Anaconda-Platform/nb_conda)

1. Go to [Google REST API quick tutorial for python](https://developers.google.com/drive/v3/web/quickstart/python).
2. Once you are there, create a project, add credentials and **download secret json file**.
3. You do not have to copy ```quickstart.py``` since it sits next to this notebook! But you can...
4. Run ```quickstart.py```. This creates ```~/.credentials```.
5. Keep your **secret json file**!

### ```~/.credentials``` - Drive
### **secret json file** - pygsheets

#### First things first

In [ ]:
import pandas as pd

#### pygsheets are great for editing sheets and it has pandas integration!

In [ ]:
import pygsheets as pyg

#### this is required by Google for playing with gdrive

In [ ]:
from apiclient import discovery

In [ ]:
from oauth2client.file import Storage

In [ ]:
import httplib2

#### do not laugh! free API has some access limitations. it may be useful to make some stops between iterations or something...

In [ ]:
import time

#### where is your **secret json file?**. It is best to keep it in the same directory as your notebook

In [ ]:
secret_json = ""

### **FULL PATH** to your ```~/.credentials/drive-python-quickstart.json``` file.

In [ ]:
credential_path = ""

## Authorize pyg and set retry limit
#### This cell should propt you as any other app for visiting link, then copying success code back here. Be not afraid.

In [ ]:
gc = pyg.authorize(outh_file=secret_json,
                   outh_nonlocal=True,
                   retries=5)

## Authorize Drive

In [ ]:
store = Storage(credential_path)

In [ ]:
credentials = store.get()

In [ ]:
http = credentials.authorize(httplib2.Http())

In [ ]:
service = discovery.build("drive", "v3", http=http)

#### mind if len(results["files"]) > 1000 which is max per page you have to take care of pageToken.
#### pageToken can be retrieved by 
#### ```res = service.files().list(pageSize=1000, fields="nextPageToken, files").execute()```
#### ```token = res["nextPageToken"]```
#### and then passed as arg like:
#### ```service.files().list(pageSize=1000, pageToken=token, fields="nextPageToken, files").execute()```

## Test if it works

In [ ]:
results = service.files().list(pageSize=1000).execute()

#### how many files are on your Drive?

In [ ]:
len(results["files"])

#### check if folder ID on your Drive matchs with this result

In [ ]:
my_gdrive_folder_name = ""

In [ ]:
gdrive_out_dir = [i["id"] for i in results["files"] if i["name"] == my_gdrive_folder_name][0]

#### ```gdrive_out_dir``` should match with the last part of the browser's address pointing to your folder

In [ ]:
gdrive_out_dir

# Let's make some serious sheet on Drive!

In [ ]:
sheet_name = "serious sheet"

In [ ]:
df = pd.read_csv("./W303-cog7-day42.csv",
                 index_col=["Gene", "Sample", "Number"])

In [ ]:
df

In [ ]:
sh = gc.create(sheet_name, parent_id=gdrive_out_dir)

In [ ]:
for i in set(df.index.get_level_values(level=0)):
    time.sleep(3)
    wks = sh.add_worksheet(i)
    wks.set_dataframe(df.xs(i),
                      (1, 1),
                      copy_index=True,
                      fit=True)
sh.del_worksheet(sh.worksheet_by_title("Sheet1"))